In [1]:
# import re
# import collections
# import pdfplumber
# import os
# import json

# from classBook import Book


# dataDir = "../data/"
# dataName = "Deep Learning.pdf"

In [2]:
import json
import os
import re
import pdfplumber

dataDir = "../data/"
dataName = "Deep Learning.pdf"


class Book:
    num_pages = -1
    book_end_page = "735"
    page_offset = 15

    def __init__(self, name=dataName, dataDir=dataDir):
        self.name = name
        self.dataDir = dataDir
        self.pdf = self.loader()

    def loader(self):
        print("Reading book: ", self.name, "from directory: ", self.dataDir)
        try:
            pdf = pdfplumber.open(os.path.join(self.dataDir, self.name))
        except:
            print("Error: File not found")
            return None
        self.num_pages = len(pdf.pages)
        print("Book loaded successfully")
        print("Number of pages: ", self.num_pages)
        return pdf

    def close(self):
        self.pdf.close()

    def readPage(self, page=-1):
        if page == -1:
            return self.pdf.pages
        else:
            return self.pdf.pages[page - 1]

    def readPageInInterval(self, start_page, end_page, offset=page_offset):
        return self.pdf.pages[start_page - 1 + offset : end_page - 1 + offset]

    def searchStrInPage(self, page, str):
        page_text = self.readPage(page)
        return page_text.extract_text().lower().find(str.lower())

    def getToc(self):
        pages = []
        for i in range(8):
            if self.searchStrInPage(i, "Contents") != -1:
                pages.append(i)
        # reg expression to match '6 Deep Feedforward Networks 168'
        pattern_chapter = re.compile(r"(\d+)\s+(.*)\s+(\d+)")
        # match '6.1 Example: Learning XOR . . . . . . . . . . . . . . . . . . . . . . . 171',
        pattern_section = re.compile(
            r"(\d+)\.(\d+)\s+([\?\,\'\’\(\)a-zA-Z\:\s\-]+)\s+.*\s+(\d+)"
        )

        # save to dict
        toc = {}
        for page in pages:
            page_text = self.readPage(page)
            text = page_text.extract_text()
            lines = text.split("\n")
            for line in lines:
                match_chapter = pattern_chapter.match(line)
                match_section = pattern_section.match(line)
                if match_chapter:
                    chapter = {
                        "chapter": match_chapter.group(1),
                        "title": match_chapter.group(2),
                        "page": match_chapter.group(3),
                    }
                elif match_section:
                    section = {
                        "chapter": match_section.group(1),
                        "section": match_section.group(2),
                        "title": match_section.group(3),
                        "page": match_section.group(4),
                    }
                    if chapter["chapter"] not in toc:
                        toc[chapter["chapter"]] = {
                            "title": chapter["title"],
                            "page": chapter["page"],
                            "sections": [],
                        }
                    toc[chapter["chapter"]]["sections"].append(section)

        # add end page
        for chapter in toc:
            try:
                toc[chapter]["end_page"] = toc[str(int(chapter) + 1)]["page"]
            except:
                toc[chapter]["end_page"] = self.book_end_page
            for section in toc[chapter]["sections"]:
                try:
                    section["end_page"] = toc[chapter]["sections"][
                        int(section["section"])
                    ]["page"]
                except:
                    section["end_page"] = toc[chapter]["end_page"]
        # write to json
        with open(os.path.join(self.dataDir, "toc.json"), "w") as f:
            json.dump(toc, f, indent=4)
        return len(toc)

    def loadToc(self):
        with open(os.path.join(self.dataDir, "toc.json"), "r") as f:
            toc = json.load(f)
        return toc

    def getChapter(self, chapter):
        toc = self.loadToc()
        page = toc[chapter]["page"]
        end_page = toc[chapter]["end_page"]
        return self.readPageInInterval(int(page), int(end_page))

In [3]:
book = Book(dataName)
# book.getToc()
# toc = book.loadToc()

Reading book:  Deep Learning.pdf from directory:  ../data/
Book loaded successfully
Number of pages:  800


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
import IPython
from pyvis.network import Network

c:\Users\HP\.conda\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = "", "", "", ""
    text = text.strip()
    current = "x"
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = "t"
            if relation != "":
                relations.append(
                    {
                        "head": subject.strip(),
                        "type": relation.strip(),
                        "tail": object_.strip(),
                    }
                )
                relation = ""
            subject = ""
        elif token == "<subj>":
            current = "s"
            if relation != "":
                relations.append(
                    {
                        "head": subject.strip(),
                        "type": relation.strip(),
                        "tail": object_.strip(),
                    }
                )
            object_ = ""
        elif token == "<obj>":
            current = "o"
            relation = ""
        else:
            if current == "t":
                subject += " " + token
            elif current == "s":
                object_ += " " + token
            elif current == "o":
                relation += " " + token
    if subject != "" and relation != "" and object_ != "":
        relations.append(
            {"head": subject.strip(), "type": relation.strip(), "tail": object_.strip()}
        )
    return relations

In [11]:
class KB:
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [12]:
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(
        text, max_length=512, padding=True, truncation=True, return_tensors="pt"
    )
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 216,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3,
    }
    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

In [22]:
test = [page.extract_text().replace("\n", " ") for page in book.getChapter("1")]
# save to txt
# with open(os.path.join(dataDir, "test.txt"), "w") as f:
#     f.write(test)


kb = from_small_text_to_kb(test, verbose=True)
kb.print()

Num tokens: 512
Relations:
  {'head': 'intelligent software', 'type': 'use', 'tail': 'artificial intelligence'}
  {'head': 'Galatea', 'type': 'instance of', 'tail': 'legendary figure'}
  {'head': 'Pygmalion', 'type': 'instance of', 'tail': 'legendary figure'}
  {'head': 'Deep Blue', 'type': 'instance of', 'tail': 'AI'}
  {'head': 'Deep Blue', 'type': 'uses', 'tail': 'AI deep learning'}
  {'head': 'Deep Blue', 'type': 'use', 'tail': 'AI deep learning'}
  {'head': 'algorithm', 'type': 'part of', 'tail': 'computer science'}
  {'head': 'computer science', 'type': 'has part', 'tail': 'artificial intelligence'}
  {'head': 'artificial intelligence', 'type': 'part of', 'tail': 'computer science'}
  {'head': 'algorithm', 'type': 'part of', 'tail': 'artificial intelligence'}
  {'head': 'artificial intelligence', 'type': 'has part', 'tail': 'algorithm'}
  {'head': 'au- toencoder', 'type': 'instance of', 'tail': 'algorithm'}
  {'head': 'au- toencoder', 'type': 'subclass of', 'tail': 'Autoencoder'}